In [ ]:
import torch
import torchaudio
import torch.nn import nn
import torch.nn.functional as F
from torch.utils.data import Dataset
import numpy as np
from tqdm import tqdm
import os


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("f{device} used")

In [2]:
epochs = 40
lr = 0.001
batch_size = 4

In [ ]:
class AudioData(Dataset):
    def __init__(self, root):
        self.directory_path = root
        self.classes = os.listdir(self.directory_path)
        print(self.classes)
        self.data_path = []
        self.labels = []

        for root, dirs, files in os.walk(self.directory_path):
            for file in files:
                if file != ".DS_Store":
                    label = os.path.basename(root)


    def __getitem__(self, index):
        
            self.data_path = self.directory_path[index]
            
            


In [ ]:
class M5(nn.Module):
    def __init__(self, n_input=1, n_output=35, stride=16, n_channel=32):
        super().__init__()
        self.conv1 = nn.Conv1d(n_input, n_channel, kernel_size=80, stride=stride)
        self.bn1 = nn.BatchNorm1d(n_channel)
        self.pool1 = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(n_channel, n_channel, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(n_channel)
        self.pool2 = nn.MaxPool1d(4)
        self.conv3 = nn.Conv1d(n_channel, 2 * n_channel, kernel_size=3)
        self.bn3 = nn.BatchNorm1d(2 * n_channel)
        self.pool3 = nn.MaxPool1d(4)
        self.conv4 = nn.Conv1d(2 * n_channel, 2 * n_channel, kernel_size=3)
        self.bn4 = nn.BatchNorm1d(2 * n_channel)
        self.pool4 = nn.MaxPool1d(4)
        self.fc1 = nn.Linear(2 * n_channel, n_output)


    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = F.avg_pool1d(x, x.shape[-1])
        x = x.permute(0, 2, 1)
        x = self.fc1(x)
        x = F.softmax(x, dim=1)
        return x




In [ ]:
optimizer = torch.optim.Adam(mo)

In [ ]:

# train
model.train()

for epoch in range(epochs):
    train_loss = 0.0
    train_acc = 0.0
    for audios, labels in tqdm(train_data_loader):
        audios, labels = audios.to(device), labels.to(device)
        labels_onehot = torch.nn.functional.onehot(labels, num_classes=10).type
        optimizer.zero_grad()

        preds = model(audios)

        loss = loss_function(preds, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss
        train_acc += model.accuracy(preds, labels)
    
    total_loss = train_loss / len(train_data_loader)
    total_acc = train_acc / len(train_data_loader)

    print(f"Epoch: {epoch}, Loss: {total_loss}, Acc: {total_acc}")
     